In [2]:
from IPython.display import Image, display

import os
import autogen
from autogen.coding import LocalCommandLineCodeExecutor
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv

load_dotenv()

config_list = [
    {
        'model': 'gpt-4o',
        'api_key':  os.getenv("OPENAI_API_KEY"),
        'tags': ['tool', 'gpt-4'],
    },
    {
        'model': 'gemini-1.5-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.5-flash',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['tool', 'gemini'],
    },
    {
        'model': 'gemini-1.0-pro',
        'api_key': os.getenv("GEMINI_API_KEY"),
        'api_type': 'google',
        'tags': ['gemini'],
    }
]

import json
config_list_file_name = ".config_list"
with open(config_list_file_name, "w") as file:
    json.dump(config_list, file, indent=4)

gpt_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gpt-4"]})
gpt_llm_config = {"config_list": gpt_config_list, "timeout": 120}

gemini_config_list = autogen.config_list_from_json(config_list_file_name, filter_dict={"tags": ["gemini"]})
gemini_llm_config = {"config_list": gemini_config_list, "timeout": 120}

In [11]:
import os
import json
import autogen
from typing import List, Dict, Any, Optional
from utils import search_arxiv, create_pdf

# Create a UserProxyAgent with a better configuration for function execution
user_proxy = autogen.UserProxyAgent(
    name="User",
    system_message="A human user who needs a technical review document.",
    code_execution_config={
        "last_n_messages": 3, 
        "work_dir": "paper_review",
        "use_docker": False  # Set to True if you want to use Docker for isolation
    },
    human_input_mode="NEVER"
)

# Researcher agent with improved system message
researcher = autogen.AssistantAgent(
    name="Researcher",
    system_message="""You are a researcher specialized in finding relevant research papers.
    You're expert at retrieving and analyzing scientific publications.
    When asked to search for papers.
    Format your findings as a structured report with clear sections and paper summaries.
    """,
    llm_config={"config_list": gemini_config_list, "timeout": 120}
)

# Writer agent
writer = autogen.AssistantAgent(
    name="Writer",
    system_message="""You are a technical writer specialized in creating comprehensive review documents.
    You take research papers and organize them into a structured review document.
    Your reviews include:
    1. Executive summary
    2. Introduction to the field
    3. Analysis of current research directions
    4. Detailed summaries of key papers
    5. Comparison of methodologies
    6. Future research directions
    7. Conclusion
    8. References (in IEEE format)
    Your writing is clear, precise, and technical but accessible.""",
    llm_config={"config_list": gemini_config_list, "timeout": 120}
)

# PDF creator agent
pdf_creator = autogen.AssistantAgent(
    name="PDFCreator",
    system_message="""You are an expert in creating PDF documents using Python.
    You take the final review document content and format it into a professional PDF.
    You may install required Python modules i.e. fpdf2 and so on and implement required python function complete the job.""",
    llm_config={"config_list": gemini_config_list, "timeout": 120}
)

# Create a function to run the entire workflow
def run_review_workflow(topic: str) -> str:
    """
    Execute the complete workflow to generate a technical review document.
    
    Args:
        topic: The research topic to generate a review for
        
    Returns:
        Path to the generated PDF
    """
    # Create the working directory
    # os.makedirs("paper_review", exist_ok=True)
    
    # Copy utils.py to the working directory to ensure it's accessible
    # import shutil
    # shutil.copy("utils.py", os.path.join("paper_review", "utils.py"))
    
    # Create a group chat
    groupchat = autogen.GroupChat(
        agents=[user_proxy, researcher, writer, pdf_creator],
        messages=[],
        max_round=20  # Increase max rounds to allow for more interaction
    )
    
    # Create a group chat manager WITH an LLM config (this was missing before)
    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config={"config_list": gemini_config_list}
    )
    

    # Start the chat with clear instructions
    user_proxy.initiate_chat(
        manager,
        message=f"""
        I need a comprehensive technical review document on "{topic}".
        
        Follow these steps exactly:
        
        1. Researcher: Search for the most relevant papers on this topic by writing required python code.
           You may use Python modules i.e. arxiv to search the publications by topic. Don't implement the version that has API key dependency. 
        2. Researcher: Analyze the results and identify the 5-7 most important papers returned by the impemented function.
           You may need to debug the function and refactor it until you get valid responses.
        
        3. Writer: Create a comprehensive technical review based on these papers.
        
        4. PDFCreator: Implement create_pdf function and execute it. You may need to debug until it returns valid PDF.
           Provide the the final review document retrieved from the Writer to the create_pdf function. 
           File name should be "{topic.replace(' ', '_')}_review.pdf"
           
        5. The final output should be a PDF file saved in the working directory.
        """
    )
    
    # Return the expected filename
    return f"{topic.replace(' ', '_')}_review.pdf"

# Usage example
if __name__ == "__main__":
    # Set your OpenAI API key as an environment variable before running
    # os.environ["OPENAI_API_KEY"] = "your-api-key"
    
    topic = "quantum machine learning"
    pdf_path = run_review_workflow(topic)
    print(f"Workflow completed. Review document should be at: {pdf_path}")

User (to chat_manager):


        I need a comprehensive technical review document on "quantum machine learning".
        
        Follow these steps exactly:
        
        1. Researcher: Search for the most relevant papers on this topic by writing required python code.
           You may use Python modules i.e. arxiv to search the publications by topic. Don't implement the version that has API key dependency. 
        2. Researcher: Analyze the results and identify the 5-7 most important papers returned by the impemented function.
           You may need to debug the function and refactor it until you get valid responses.
        
        3. Writer: Create a comprehensive technical review based on these papers.
        
        4. PDFCreator: Implement create_pdf function and execute it. You may need to debug until it returns valid PDF.
           Provide the the final review document retrieved from the Writer to the create_pdf function. 
           File name should be "quantum_mac